In [232]:
from bs4 import BeautifulSoup as bs
import requests
import re
from pprint import pprint

### 1 Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта hh.ru

In [233]:
def query(link,params={}):
    """
    Функция содержит общие параметры необходимые для запроса. 
    link – хранит ссылку на сайт, на который адресован запрос.
    params  - необязательный параметр, содержит передаваемые параметры, параметры можно передать и в ссылке link, а params оставить не заполненным.
    Возвращает функция DOM дерево.
    """
    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
          'Accept':'*/*'}
    if papametr:
        response = requests.get(link,headers=header,params=params)
        print(response.url)
        response = response.text
    else:
        response = requests.get(link,headers=header).text
   
    soup = bs(response,'lxml')    
    return soup



def getSalary(s):
    
    """
    Функция обрабатывает строки содержащие величину заработной платы и возвращает dist, содержащий цены в структурированном виде.
    """
    
    obj = {"min":None,
           "max":None,
           "currency_unit":None
          }
    if not s:
        return obj
    q = re.sub('\s*', "", s)
    result = re.findall(r'([^0-9]*)(?:(\d*)\W)*(\d+)(\w+)', q)
    if result:
        result = result[0]
        obj["currency_unit"] = result[3].upper()
        if result[0].upper() == "от".upper():        
            obj["min"] = float(result[2])
        elif result[0].upper() == "до".upper():      
            obj["max"] = float(result[2])
        elif result[0] == "":
            obj["min"] = float(result[1])
            obj["max"] = float(result[2])           
    return obj
    


In [243]:
main_link = 'https://murmansk.hh.ru/search/vacancy?text=data+scientist'
base_link = 'https://murmansk.hh.ru'
params = {'L_is_autosearch':'false',
            'clusters':'true',
            'enable_snippets':'true',           
            'page':0}

In [244]:
w = []
while True:
    soup = query(link=main_link,params=params)
    vacancy_serpclass = soup.find('div', {'class':'vacancy-serp'})
    vacancies = vacancy_serpclass.select(".vacancy-serp-item")
    pprint(len(vacancies))    
    button = soup.find('a', {"data-qa":"pager-next"})#кнопка ДАЛЕЕ    
    
    for item in vacancies:
        a = item.find('a', {"data-qa":"vacancy-serp__vacancy-title"})
        site = item.find('a', {"data-qa":"vacancy-serp__vacancy-employer"})        
        salary = item.find('span', {"data-qa":"vacancy-serp__vacancy-compensation"})
        salary = salary.text if salary else None
        salary = getSalary(salary)
        obj = {'specialty':a.string,
               'salary_max':salary['max'],
               'salary_min':salary['min'],
               'salary_unit':salary['currency_unit'],
               'link':a.get("href"),
               'site':base_link+site.get("href") if site else None,
               'site_job':'HH'}        
        w.append(obj)
        
    if not button:  #проверяю наличие, кнопку ДАЛЕЕ
        break
    
    params["page"] = params["page"] + 1

https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=0
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=1
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=2
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=3
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=4
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=5
50
https://murmansk.hh.ru/search/vacancy?text=data+scientist&L_is_autosearch=false&clusters=true&enable_snippets=true&page=6
23


### 2 Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru

In [245]:
main_link = 'https://russia.superjob.ru/vakansii/programmist-python.html'
base_link = 'https://russia.superjob.ru'
params = {'page':1}
soup = query(link=main_link,params=params)

https://russia.superjob.ru/vakansii/programmist-python.html?page=1


In [256]:
while True:
    soup = query(link=main_link,params=params)
    vacancies = soup.select(".iJCa5.f-test-vacancy-item._1fma_._1JhPh._2gFpt._1znz6._2nteL")
    pprint(len(vacancies)) 
    button = soup.select('.icMQ_._1_Cht._3ze9n.f-test-button-dalshe.f-test-link-Dalshe')#кнопка ДАЛЕЕ   
    for item in vacancies:
        a = item.select(".icMQ_._6AfZ9")[0]        
        site = item.select('.icMQ_._205Zx')
        site = site[0] if site else None        
        salary = item.select('._3mfro._2Wp8I.PlM3e._2JVkc._2VHxz')
        salary = salary[0].text if salary else None
        salary = getSalary(salary)        
        obj = {'specialty':a.text,
               'salary_max':salary['max'],
               'salary_min':salary['min'],
               'salary_unit':salary['currency_unit'],
               'link':base_link+a.get("href"),
               'site':base_link+site.get("href") if site else None,
               'site_job':'superjob'}
        w.append(obj)
    if not button:#проверяю наличие, кнопку ДАЛЕЕ
        break
    params["page"] = params["page"] + 1

https://russia.superjob.ru/vakansii/programmist-python.html?page=2
9


In [259]:
import pandas as pd
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.max_rows', 400)
 
vacancies = pd.DataFrame(w)

In [260]:
vacancies

,specialty,salary_max,salary_min,salary_unit,link,site,site_job
0,Специалист по углубленной аналитике данных (Data scientist),NaN,NaN,None,https://murmansk.hh.ru/vacancy/37732722?query=data%20scientist,https://murmansk.hh.ru/employer/778329?dpt=778329-778329-01,HH
1,Senior Data Engineer / Python,NaN,NaN,None,https://murmansk.hh.ru/vacancy/37762953?query=data%20scientist,https://murmansk.hh.ru/employer/841087,HH
2,Data Scientist,180000.0,120000.0,РУБ,https://murmansk.hh.ru/vacancy/37635578?query=data%20scientist,https://murmansk.hh.ru/employer/3218306,HH
3,Python Data Scientist,NaN,150000.0,РУБ,https://murmansk.hh.ru/vacancy/37521387?query=data%20scientist,https://murmansk.hh.ru/employer/5552,HH
4,Аналитик-математик (Data scientist),100000.0,80000.0,РУБ,https://murmansk.hh.ru/vacancy/37698748?query=data%20scientist,https://murmansk.hh.ru/employer/2633363,HH
5,Data Scientist,NaN,130000.0,РУБ,https://murmansk.hh.ru/vacancy/37569615?query=data%20scientist,https://murmansk.hh.ru/employer/2558781,HH
6,Data Scientist,NaN,NaN,None,https://murmansk.hh.ru/vacancy/37559909?query=data%20scientist,https://murmansk.hh.ru/employer/78638,HH
7,Computer Vision / Data scientist CV,NaN,NaN,None,https://murmansk.hh.ru/vacancy/37715784?query=data%20scientist,https://murmansk.hh.ru/employer/669940,HH
8,Математик-аналитик (data scientist / data analyst),NaN,85000.0,РУБ,https://murmansk.hh.ru/vacancy/37327686?query=data%20scientist,https://murmansk.hh.ru/employer/2954482,HH
9,Data Scientist,NaN,NaN,None,https://murmansk.hh.ru/vacancy/37662353?query=data%20scientist,https://murmansk.hh.ru/employer/1988556,HH
